In [69]:
import pandas as pd
import json
pd.options.display.max_rows = 999

In [70]:
dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_860m/'
jsonDir = '/Users/rachelanderson/Dropbox (Princeton)/Tax Equity Code/Analyses/'

In [71]:
#import data
data = pd.ExcelFile(dataDir + 'february_generator2020.xlsx')
df = data.parse(sheetname=data.sheet_names[0], skiprows=1)

#clean column names
df.columns = [x.lower().replace(" ","_") for x in df.columns]
df.columns = [x.split('_(mw)')[:-1][0] if "(mw)" in x else x for x in df.columns]

In [ ]:
x

In [32]:
df.groupby('entity_name').nameplate_capacity.sum().sort_values(ascending=False).reset_index().head(10)

,entity_name,nameplate_capacity
0,Tennessee Valley Authority,35792.9
1,Florida Power & Light Co,28635.7
2,"Duke Energy Carolinas, LLC",24090.6
3,Virginia Electric & Power Co,23066.4
4,Exelon Nuclear,22817.3
5,Georgia Power Co,20719.0
6,U S Bureau of Reclamation,14690.2
7,Duke Energy Progress - (NC),14389.2
8,USACE Northwestern Division,13111.4
9,Alabama Power Co,12912.7


In [54]:
df[(df['energy_source_code']=='SUN') & (df['nameplate_capacity'] >= 100 )].sort_values(by='nameplate_capacity',ascending=False).reset_index()

,index,entity_id,entity_name,plant_id,plant_name,sector,plant_state,generator_id,unit_code,nameplate_capacity,...,planned_derate_of_summer_capacity,planned_uprate_year,planned_uprate_month,planned_uprate_of_summer_capacity,county,latitude,longitude,google_map,bing_map,balancing_authority_code
0,16039,56722,AV Solar Ranch 1 LLC,57378.0,AV Solar Ranch One,IPP Non-CHP,CA,AVSR,NaN,253.0,...,,,,,Los Angeles,34.780278,-118.42440,Map,Map,CISO
1,20639,61230,"CD Arevon USA, Inc.",61202.0,Mount Signal Solar Farm V,IPP Non-CHP,CA,MTSG3,NaN,252.3,...,,,,,Imperial,32.678805,-115.56720,Map,Map,CISO
2,21503,61515,"Phoebe Energy Project, LLC",61906.0,Phoebe Solar,IPP Non-CHP,TX,PHOEB,NaN,250.0,...,,,,,Winkler,31.843422,-102.86830,Map,Map,ERCO
3,16654,61230,"CD Arevon USA, Inc.",57859.0,Moapa Southern Paiute,IPP Non-CHP,NV,1,NaN,250.0,...,,,,,Clark,36.529950,-114.77030,Map,Map,NEVP
4,21803,63048,Misae Lessee LLC,62249.0,Misae Solar,IPP Non-CHP,TX,77777,NaN,240.0,...,,,,,Childress,34.380532,-100.09960,Map,Map,ERCO
5,19339,17650,Southern Power Co,59939.0,RE Tranquillity,IPP Non-CHP,CA,TQ,NaN,205.3,...,,,,,Fresno,36.581365,-120.40450,Map,Map,CISO
6,19440,59777,"Buckthorn Westex, LLC",60044.0,Buckthorn Westex,IPP Non-CHP,TX,BKTH1,NaN,202.0,...,,,,,Pecos,30.575520,-102.55130,Map,Map,ERCO
7,21523,61533,Techren Solar II LLC,61930.0,Techren Solar II LLC,IPP Non-CHP,NV,TECH2,NaN,200.0,...,,,,,Clark,35.867258,-114.95640,Map,Map,NEVP
8,18867,59260,Wright Solar Park LLC,59525.0,Wright Solar Park,IPP Non-CHP,CA,FRWSP,NaN,200.0,...,,,,,Merced,37.009444,-120.95720,Map,Map,CISO
9,19622,17650,Southern Power Co,60233.0,RE Garland,IPP Non-CHP,CA,PV2,NaN,185.0,...,,,,,Kern,34.825267,-118.52500,Map,Map,CISO


In [45]:
df[df['plant_name'].str.contains("Misae").fillna(False)]

,entity_id,entity_name,plant_id,plant_name,sector,plant_state,generator_id,unit_code,nameplate_capacity,net_summer_capacity,...,planned_derate_of_summer_capacity,planned_uprate_year,planned_uprate_month,planned_uprate_of_summer_capacity,county,latitude,longitude,google_map,bing_map,balancing_authority_code
21803,63048,Misae Lessee LLC,62249.0,Misae Solar,IPP Non-CHP,TX,77777,NaN,240.0,240,...,,,,,Childress,34.380532,-100.0996,Map,Map,ERCO


In [ ]:
def id_parent_comp(df, utility_var, address_var, utility_dict):
    #for d in utility_dicts:
        
    df['parent_utility']=df[utility_var].map(utility_dict)
    
    df['parent_utility']=df['parent_utility'].fillna(df[utility_var])
    
    # assign same address to common parent 
    df[df[address_var].str.contains('1414 harbour way')]['parent_utility'] = "Solar Star"
    df[df[address_var].str.contains('700 universe')]['parent_utility'] = 'NextEra'
    
    
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "Duke Energy" if "Duke Energy" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "NextEra" if "NextEra" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "NRG" if "NRG" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "Solar Star" if "Solar Star" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "NRG" if "Agua Caliente Solar" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "NextEra" if "Blythe Solar" in x else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "8me" if "8me" in x.lower() else x)
    df['parent_utility'] = df['parent_utility'].apply(lambda x: "Exelon" if "AV Solar" in x else x)
    
    return df